In [3]:
import ccxt
import time

# Replace with your actual API keys for Bybit and Bitget
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Connect to Bitget API
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# Settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
LEVERAGE = 5  # Adjustable leverage
INVESTMENT_USDT = 1000  # Example amount to use per trade

# Define exchange-specific fees
TRADING_FEES = {
    'Bybit': 0.001,   # 0.1% fee
    'Bitget': 0.0008  # 0.08% fee
}

exchanges = {'Bybit': bybit, 'Bitget': bitget}

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        for symbol, market in markets.items():
            if exchange_name == "Bitget":
                if not market.get('spot', False):
                    continue
            if '/USDT' in symbol and ':USDT' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception as e:
                    print(f"⚠️ Skipping {symbol} on {exchange_name} due to error: {e}")
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        if symbol in exchange.load_markets():
            ticker = exchange.fetch_ticker(symbol)
            return ticker.get("last", None)
    except Exception as e:
        print(f"❌ Error fetching price for {symbol} on {exchange}: {e}")
    return None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol):
    spot_price = get_price(exchange, symbol)
    futures_symbol = symbol.replace('/USDT', '/USDT:USDT')

    if futures_symbol not in exchange.load_markets():
        return

    futures_price = get_price(exchange, futures_symbol)
    if not spot_price or not futures_price:
        return

    fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (fee * 2)

    if net_profit >= SPREAD_THRESHOLD:
        token_quantity = (INVESTMENT_USDT * LEVERAGE) / spot_price
        print(f"🔍 Arbitrage Opportunity on {exchange_name}: {symbol}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        print(f"    📦 Token Quantity (with {LEVERAGE}x leverage): {token_quantity:.4f}")
        print(f"    ⚡ Execute arbitrage on {exchange_name}")
        print("-" * 50)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                for token in tradable_tokens:
                    check_intra_exchange_arbitrage(exchange, exchange_name, token)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()


❌ Error fetching tradable tokens from Bybit: bybit {"retCode":10002,"retMsg":"invalid request, please check your server timestamp or recv_window param. req_timestamp[1742548339017],server_timestamp[1742548344109],recv_window[5000]","result":{},"retExtInfo":{},"time":1742548344109}
❌ No tradable tokens found on Bybit. Retrying in 10 seconds...
✅ Found 542 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...
🔍 Arbitrage Opportunity on Bitget: JELLYJELLY/USDT
    ✅ Spot Price: 0.02124 USDT
    ✅ Futures Price: 0.02142 USDT
    ✅ Spread: 0.8475%
    ✅ Net Profit After Fees: 0.6875%
    📦 Token Quantity (with 5x leverage): 235404.8964
    ⚡ Execute arbitrage on Bitget
--------------------------------------------------
✅ Cycle complete. Waiting for next check...
❌ Error fetching tradable tokens from Bybit: bybit {"retCode":10002,"retMsg":"invalid request, please check your server timestamp or recv_window param. req_timestamp[1742548606528],server_timestamp[1742548611